# 1. data normalized

In [103]:
import numpy as np
import torch

In [104]:
torch.set_printoptions(edgeitems = 2, threshold = 50)

👆

> 更改打印设置

这行代码是设置PyTorch的打印选项。

`torch.set_printoptions(edgeitems=2, threshold=50)`这行代码做了两件事：

1. `edgeitems=2`：这个参数设置了当打印张量时，每个维度的开始和结束部分打印多少个元素。在这个例子中，每个维度的开始和结束部分都会打印2个元素。

2. `threshold=50`：这个参数设置了张量的总元素数量超过多少时，张量会被“收缩”为一个更紧凑的格式。在这个例子中，如果张量的总元素数量超过50，那么张量就会被收缩为一个更紧凑的格式。

这些设置可以帮助你更好地查看和理解大型张量的数据。

In [105]:
import imageio.v2 as imageio

👆

```python
>>> import imageio
>>> img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')
/tmp/ipykernel_1605/2656173173.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')
```

对此警告，它告诉你在ImageIO版本3中，`imread`函数的行为将会改变。

警告的具体内容是：从ImageIO v3开始，这个函数的行为将切换为`iio.v3.imread`的行为。如果你想保持当前的行为（并消除这个警告），你可以使用`import imageio.v2 as imageio`或直接调用`imageio.v2.imread`。

通过以下方式可以消除这个警告：

```python
import imageio.v2 as imageio

img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')
```

这样，你的代码就会继续使用ImageIO v2版本的`imread`函数，而不会在升级到ImageIO v3后改变行为。


In [106]:
img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')
img_arr.shape

(720, 1280, 3)

👆

```bash
(720, 1280, 3)
```

对此明显的, 最后的`3`是色彩空间, 所以其维度顺序与之前所见的明显是不同的, 由此可以了解到, 前两个哪个是高度, 哪个是宽度也是不清楚的.

> 另外, 在部分图片中还会有第4维度, 即不透明度

In [107]:
img_arr.dtype

dtype('uint8')

In [108]:
img = torch.from_numpy(img_arr)

In [109]:
out = img.permute(2, 0, 1) # 用于将三个维度换位
img_arr.shape

(720, 1280, 3)

In [110]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype = torch.uint8)

👆

`batch`准备了了三个图片, 每个图片为一个`3*256*256`的张量.

---

接下来就是读进3个图片, 即将三个图片读入`batch`中.

👇

In [111]:
import os

In [112]:
data_dir = '../data/p1ch4/image-cats/'

In [113]:
filenames = [name for name in os.listdir(data_dir) 
             if os.path.splitext(name)[-1] == '.png']

👆

1. `os.listdir(data_dir)`：这个函数会返回指定目录下的所有文件和目录名的列表。

2. `os.path.splitext(name)`：这个函数会将文件名和扩展名分开。它返回一个元组，元组的第一个元素是文件名，第二个元素是扩展名。例如，`os.path.splitext('example.png')`会返回`('example', '.png')`。

3. `os.path.splitext(name)[-1] == '.png'`：这个表达式会检查文件的扩展名是否为`.png`。如果是，就返回True，否则返回False。

4. `[name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png']`：这是一个列表推导式，它会遍历`data_dir`目录下的所有文件和目录名，如果文件的扩展名为`.png`，就将文件名添加到列表中。

所以，这段代码的结果是一个列表，列表中的元素是`data_dir`目录下所有以`.png`为后缀的文件的文件名。

In [114]:
filenames

['cat1.png', 'cat2.png', 'cat3.png']

In [115]:
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename)) # 把图片文件读到img_arr里边 
    img_t = torch.from_numpy(img_arr) # 转tensor
    img_t = img_t.permute(2, 0, 1) # 换位置
    img_t = img_t[:3] # 为了保险起见, 因为部分图片有第四维度, 所以只取前三个
    batch[i] = img_t 

👆

### `enumerate`

`enmuerte`函数的作用是将一个可迭代对象, 如下:

```python
for i, filename in enumerate(filenames):
    print(i, filename)
```

返回的结果是:

```python
0 cat1.png
1 cat2.png
2 cat3.png
```

---

如果把`index`去掉:

```python
for filename in filenames:
    print(filename)
```

返回的结果是:

```python
cat1.png
cat2.png
cat3.png
```

---

也就是, `enumerate`函数的作用是将一个可迭代对象, 返回一个`index`和`value`的元组, 即如果需要`index`就要用`enumerate0`.

### `img_t[:3]`

这里指的前三个维度不是指`img_t.shape`的前三个数, 而是指的是对于`img_t`的第一个维度的前三个维度.

- 第一个维度即RGB的维度, 也就是`3*256*256`中的`3`
- 而部分图片中还会有第4维度, 即不透明度
- 由此只取前三个即`RGB`, 而不要不透明度

也即是:

当`img_t.shape`的第一个维度大于等于3时, 取前三个维度, 否则取全部.

```python
>>> img_t.shape
torch.Size([4, 256, 256])
>>> img_t[:3].shape
torch.Size([3, 256, 256])
```

### `batch[i]`

`batch`在之前以近乎决定了3个维度, 也就是3个图片, 这里就让这三个图片读进了`batch`中. 也就让它们都在了同一个张量中.

In [116]:
batch = batch.float() # 模型都需要浮点型 (之前指整数型)
batch /= 255.0 # 模型更喜欢0-1, 所以除掉255

In [117]:
batch.shape

torch.Size([3, 3, 256, 256])

In [118]:
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) /std

👆

对RGB的通道进行标准化处理, 结果将是`batch[:,i].mean()`等于或近似等于0, `batch[:,i].std()`等于或近似等于1.

In [126]:
batch[:,2].mean(), batch[:,2].std()

(tensor(1.3908e-07), tensor(1.))